Dynamics of continental accretion
======

This notebook outlines the Underworld model used in the Moresi (2014) paper 'Dynamics of continental accretion'. It reproduces the initial conditions shown in Extended Data Figure 1 and 2 and the numerics required for reproduce Figure 2.

"In order to better understand the behaviour of this ancient plate mar- gin and the growth of the Australian continent, we use three-dimensional (3D) dynamic models of a subducting slab, overriding plate and mantle, building on previous work14–16. The models have a four-layer subduct- ing plate with buoyancy and rheology of each layer pre-calculated from a half-space cooling model of 80 or 120 Myr age, and they include either a weak or a strong viscoplastic overriding plate (Table 1 and Extended Data Figs 1 and 2). The simulations are best understood by viewing movies of the time evolution (Table 1)."

**References**

Moresi, L., P. G. Betts, M. S. Miller, and R. A. Cayley. 2014. “Dynamics of Continental Accretion.” Nature 508 (7495): 245–48. [doi:10.1038/nature13033](https://www.nature.com/articles/nature13033)

In [ ]:
# core UW bit
import UWGeodynamics as GEO
from UWGeodynamics import visualisation as vis
from UWGeodynamics.scaling import units as u
from UWGeodynamics.scaling import dimensionalise
from UWGeodynamics.scaling import non_dimensionalise as nd

In [ ]:
# 3rd party python modules
import math
import numpy as np
import os
import scipy

In [ ]:
import geo_model_properties as modprop
import relrho_geo_material_properties as matprop
import geo_material_properties

In [ ]:
# shortcuts for parallel wrappers
barrier = GEO.uw.mpi.barrier
rank    = GEO.rank

In [ ]:
# read in and execute internal scaling script
exec(open('geo_moresi_2014_scaling.py').read())

# scaling debug check
if rank == 0:
    print(dimensionalise(10.,u.meters)," | ", nd(100.*u.megapascal) )
    #GEO.scaling.get_coefficients()

**Setup parameters**


In [ ]:
# xRes = 256
# yRes =  96 
# zRes =  96

dim = 3

xRes = 128
yRes = 48
zRes = 48

checkpoint_restart = False
outputPath = f"output-{xRes}x{yRes}x{zRes}" if dim == 3 else f"output-{xRes}x{yRes}"

In [ ]:
outputPath = os.path.join(os.path.abspath("."), outputPath)
if rank==0:
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
barrier()

if checkpoint_restart == True:
    # you need to define these
    outputPath_restart = os.path.join(os.path.abspath("."),"output_checkPoint_restart/")
    time = 4.38067181e+03
    step = 1

**Create mesh and finite element variables**

In [ ]:
# Domain
boxLength = modprop.boxLength
boxHeight = modprop.boxHeight
boxWidth  = modprop.boxWidth

In [ ]:
# Define our vertical unit vector using a python tuple
if dim == 2:
    elementRes = (xRes, yRes)
    minCoord   = (0., -boxHeight)
    maxCoord   = (boxLength, 0.)
    z_hat      = ( 0.0, 1.0)
    
else:
    elementRes = (xRes, yRes, zRes)
    minCoord   = (0., -boxHeight, 0.)
    maxCoord   = (boxLength, 0., boxWidth)
    z_hat      = ( 0.0, 1.0, 0.0 )
    
Model = GEO.Model(elementRes = elementRes,
                  minCoord   = minCoord,
                  maxCoord   = maxCoord,
                  gravity    = z_hat,
                  outputDir  = outputPath)

### Questions: _postSoftening_

This looks like a 'hasYielded' swarm variable. It is a bool, initially set to 0 and set to 1 iff the particle has yielded at any stage. Once triggered a 1 can't heal back to 0. It enables a bifurication of the cohesion value used in the vonMises yield criterion.

I could add a swarm variable for _postSoftening_ variable in the original model of Bec's
`Model.add_swarm_variable`

or is the `_ViscosityFunction._isYielding` from `UWGeo`

**checkpoint_restart**

In [ ]:
# # if reloading from checkpoint
if checkpoint_restart == True:
    swarm.load(         outputPath_restart+'swarm.'         + str(step).zfill(5) +'.h5')
    materialIndex.load( outputPath_restart+'materialIndex.' + str(step).zfill(5) +'.h5')
    postSoftening.load( outputPath_restart+'postSoftening.' + str(step).zfill(5) +'.h5')
else:
#     swarmLayout = uw.swarm.layouts.PerCellSpaceFillerLayout( swarm=swarm, particlesPerCell=particlesPerCell )
#     swarm.populate_using_layout( layout=swarmLayout )
#     postSoftening.data[:] = 0 # initial softening flag to False
    exec(open('geo_moresi_2014_material_layout.py').read())
    exec(open('relrho_geo_material_properties.py').read())
#     if dim == 2:
#         materialIndex.data[:] = fn.branching.conditional( conditions_2d ).evaluate(Model.swarm)
#     else:
#         materialIndex.data[:] = fn.branching.conditional( conditions_3d ).evaluate(swarm)
    

In [ ]:
# set default background material parameters
Model.density = um["density"]
Model.viscosity = um["viscosity"]

In [ ]:
#variables for initialisation of shapes from file geo_moresi_2014_material_layout.py

slab_y1 = -0*nd(slab_crust)
slab_y2 = -1*nd(slab_dy)/slab_layers
slab_y3 = -2*nd(slab_dy)/slab_layers
slab_y4 = -3*nd(slab_dy)/slab_layers

backarc_dx = 1200. * u.kilometer
backarc_dy =  100. * u.kilometer
backarc_xStart = slab_xStart - backarc_dx
backarc_layers = 2
dpert = dimensionalise(pert, u.km)

backarc_y1 = -0.*nd(backarc_dy)/backarc_layers
backarc_y2 = -1.*nd(backarc_dy)/backarc_layers

trans_y1 = -0.*nd(trans_dy)
trans_y2 = -1.*nd(trans_dy)

crat_y1 = -0.*nd(craton_dy)
crat_y2 = -1.*nd(craton_dy)

In [ ]:
# general shape functions
def slabGeo(x, y, dx, dy):
    shape = [ (x,y), (x+dx,y), (x+dx,y-dy), (x,y-dy), (x-dpert,y-dy-dpert), (x-dpert,y-dpert) ]
    return GEO.shapes.Polygon(shape)

def backarcGeo(x, y, dx, dy):
    shape = [ (x,y), (x+dx,y), (x+dx-dy,y-dy), (x,y-dy)]
    return GEO.shapes.Polygon(shape)

In [ ]:
#initialising all features as shapes

op1 = slabGeo(slab_xStart, slab_y1*u.km, slab_dx, slab_dy/slab_layers)
op1_fin = Model.add_material(name="oceanic plate 1", shape=op1)

op2 = GEO.shapes.Polygon(vertices=[(nd(slab_xStart)-pert, slab_y2-pert), 
                                    (nd(slab_xStart), slab_y2),
                                    (nd(slab_xStart+slab_dx), slab_y2),
                                    (nd(slab_xStart+slab_dx), nd(slab_y2)-nd(slab_dy)/slab_layers),
                                    (nd(slab_xStart), nd(slab_y2)-nd(slab_dy)/slab_layers),
                                    (nd(slab_xStart)-pert, nd(slab_y2)-nd(slab_dy)/slab_layers-pert)])
op2_fin = Model.add_material(name = "oceanic plate 2", shape=op2)

op3 = GEO.shapes.Polygon(vertices=[(nd(slab_xStart)-pert, slab_y3-pert), 
                                    (nd(slab_xStart), slab_y3),
                                    (nd(slab_xStart+slab_dx), slab_y3),
                                    (nd(slab_xStart+slab_dx), nd(slab_y3)-nd(slab_dy)/slab_layers),
                                    (nd(slab_xStart), nd(slab_y3)-nd(slab_dy)/slab_layers),
                                    (nd(slab_xStart)-pert, nd(slab_y3)-nd(slab_dy)/slab_layers-pert)])
op3_fin = Model.add_material(name = "oceanic plate 3", shape=op3)

op4 = GEO.shapes.Polygon(vertices=[(nd(slab_xStart)-pert, slab_y4-pert), 
                                    (nd(slab_xStart), slab_y4),
                                    (nd(slab_xStart+slab_dx), slab_y4),
                                    (nd(slab_xStart+slab_dx), nd(slab_y4)-nd(slab_dy)/slab_layers),
                                    (nd(slab_xStart), nd(slab_y4)-nd(slab_dy)/slab_layers),
                                    (nd(slab_xStart)-pert, nd(slab_y4)-nd(slab_dy)/slab_layers-pert)])
op4_fin = Model.add_material(name = "oceanic plate 4", shape=op4)

In [ ]:
ba1 = backarcGeo(backarc_xStart, 0.*u.km, backarc_dx, 50.*u.km)
ba1_fin = Model.add_material(name="backArc1", shape=ba1)

ba2 = backarcGeo(backarc_xStart, -50.*u.km, backarc_dx-50*u.km, 50.*u.km)

ba2_fin = Model.add_material(name="backArc2", shape=ba2)


t1 = GEO.shapes.Box(top=Model.top,      bottom=-trans_dy/trans_layers, 
                    minX=trans_xStart, maxX=trans_xStart+trans_dx)
t1_fin = Model.add_material(name="trans1", shape=t1)

t2 = GEO.shapes.Box(top=-trans_dy/trans_layers, bottom=-trans_dy, 
                    minX=trans_xStart, maxX=trans_xStart+trans_dx)
t2_fin = Model.add_material(name="trans2", shape=t2)

c1 = GEO.shapes.Box(top=Model.top,      bottom=-craton_dy/craton_layers, 
                    minX=craton_xStart, maxX=craton_xStart+craton_dx)

c1_fin = Model.add_material(name="craton1", shape=c1)

c2 = GEO.shapes.Box(top=-craton_dy/craton_layers, bottom=-craton_dy,
                    minX=craton_xStart, maxX=craton_xStart+craton_dx)
c2_fin = Model.add_material(name="craton2", shape=c2)

bs = GEO.shapes.Polygon(vertices=[(nd(bouyStrip_xStart), 0.),
                             (nd(bouyStrip_xStart+bouyStrip_dx), 0.),
                             (nd(bouyStrip_xStart+bouyStrip_dx), 0.-nd(bouyStrip_dy)),
                             (nd(bouyStrip_xStart), 0.-nd(bouyStrip_dy))])
bs_fin = Model.add_material(name="buoyStrip", shape=bs)

# always make 2D ribbon shape, if dim == 3 it's overwritten
rib_shape = GEO.shapes.Box(top=0*u.km, bottom=-50*u.km,
                           minX=ribbon_xStart, maxX=ribbon_xStart+ribbon_dx)

if dim == 3:
    # angle we want the ribbon rotated, can be +ve or -ve
    ang = 0
    rad = np.radians(ang)

    # calculated associated half space normals
    nx = -np.cos(rad)
    nz = np.sin(rad)

    # floor
    hsp1 = GEO.shapes.HalfSpace(normal=(0.,-1.,0.), origin=(0, -50*u.km, 0.))
    # front
    hsp2 = GEO.shapes.HalfSpace(normal=(nx, 0, nz), origin=(ribbon_xStart,0.,Model.maxCoord[2]))
    # back
    hsp3 = GEO.shapes.HalfSpace(normal=(-nx, 0, -nz), origin=(ribbon_xStart+ribbon_dx,0.,Model.maxCoord[2]))
    # width
    hsp4 = GEO.shapes.HalfSpace(normal=(0, 0, -1), origin=(0.,0.,ribbon_dz))

    rib_shape = hsp1&hsp2&hsp3&hsp4

rib = Model.add_material(name="ribbon", shape=rib_shape)


op_change = Model.add_material(name="oceanic plate 1 after phase change")

lm = Model.add_material(name="lower mantle", shape= fn_y < nd(-600.0 * 10**3 * u.meter))

added_material_list = [lm, op1_fin, op2_fin, op3_fin, op4_fin, ba1_fin, ba2_fin, t1_fin, 
                       t2_fin, c1_fin, c2_fin, bs_fin, op_change, rib]

In [ ]:
figsize=(1000,300)
camera = ['rotate x 30']
boundingBox=( minCoord, maxCoord )

materialFilter = Model.materialField > 0


# ERROR with boundaringBox, maybe BUG for Okaluza
# figSwarm = vis.Figure(figsize=figsize, boundingBox=boundingBox )

# swarmPlot = vis.objects.Points(swarm, materialIndex, materialFilter, colours='gray', opacity=0.5, fn_size=2., 
#                                     discrete=True, colourBar=False, )

Fig = vis.Figure(figsize=(1200,400))

# Show single colour
# Fig.Points(Model.swarm, colour='gray', opacity=0.5, discrete=True, 
#            fn_mask=materialFilter, fn_size=2.0, colourBar=False)

# Show all glory
Fig.Points(Model.swarm, fn_colour=Model.materialField, 
           fn_mask=materialFilter, opacity=0.5, fn_size=2.0)

# Save image to disk
# Fig.save("Figure_1.png")

# Rotate camera angle
Fig.script(camera)

# Render in notebook
Fig.show()

In [ ]:
#assigning properties (density, viscosity, etc) to landforms

for i in added_material_list:
    for j in material_list:
        if i.name == j["name"]:
            print(i.name)
            i.density = j["density"]
            i.viscosity = j["viscosity"]
            c0 = j["cohesion"] if j.get('cohesion') else None
            c1 = j["cohesion2"] if j.get('cohesion2') else None
#             i.plasticity = GEO.VonMises(cohesion = c0, cohesionAfterSoftening = c1)

**Eclogite transition**

Assume that the oceanic crust transforms instantaneously and completely to eclogite at a depth of 150 km

In [ ]:
op1.phase_changes = GEO.PhaseChange((Model.y < nd(-150.*u.kilometers)),
                                          op_change.index)

In [ ]:
figsize=(1000,300)
camera = ['rotate x 30']
boundingBox=( minCoord, maxCoord )

materialFilter = Model.materialField > 1


# ERROR with boundaringBox, maybe BUG for Okaluza
# figSwarm = vis.Figure(figsize=figsize, boundingBox=boundingBox )

# swarmPlot = vis.objects.Points(swarm, materialIndex, materialFilter, colours='gray', opacity=0.5, fn_size=2., 
#                                     discrete=True, colourBar=False, )

Fig = vis.Figure(figsize=(1200,400))

# Show single colour
# Fig.Points(Model.swarm, colour='gray', opacity=0.5, discrete=True, 
#            fn_mask=materialFilter, fn_size=2.0, colourBar=False)

# Show all glory
Fig.Points(Model.swarm, fn_colour=Model.materialField, 
           fn_mask=materialFilter, opacity=0.5, fn_size=2.0)

# Save image to disk
# Fig.save("Figure_1.png")

# Rotate camera angle
Fig.script(camera)

# Render in notebook
Fig.show()

In [ ]:
# build 3D surface tracers, ie y=0. NOT available for 2D
def tracer_coords(name, minX, maxX, minZ, maxZ):
    
    xx = np.linspace(minX, maxX, max(1, int(maxX-minX))*100)
    yy = np.zeros(1)
    zz = np.linspace(minZ, maxZ, max(1, int(maxX-minX))*100)

    xx, yy, zz = np.meshgrid(xx,yy,zz)
        
    tracers = Model.add_passive_tracers(name,
                                        vertices  = [0.,0.,0.],
                                        centroids = [xx, yy, zz]) 

In [ ]:
# Over-riding plate particles
tracer_coords("orp", nd(backarc_xStart),nd(slab_xStart),
              0., nd(boxWidth))
tracer_coords("slab",nd(slab_xStart),nd(bouyStrip_xStart),
              0., nd(slab_dz))
tracer_coords("cont", nd(craton_xStart), nd(backarc_xStart),
              0., nd(boxWidth))
tracer_coords("arc", nd(ribbon_xStart), nd(ribbon_xStart+ribbon_dx),
              nd(ribbon_dz), nd(boxWidth))
tracer_coords("buoy", nd(bouyStrip_xStart), nd(slab_xStart+slab_dx),
              0., nd(slab_dz))

In [ ]:
on_grid_x = fn.math.sin(10.*np.pi*Model.x) > 0.9
on_grid_y = fn.math.sin(10.*np.pi*Model.z) > 0.9

grid_conditions = [  
                    ( on_grid_x, 1.),
                    ( on_grid_y, 1.),
                    ( True, -1.),
                  ]

if dim == 3:
    for t in Model.passive_tracers.items():
        new_var = t[1].add_variable(dataType="float", count=1)
        new_var.data[:] = fn.branching.conditional(grid_conditions).evaluate(t[1])

In [ ]:
Model.passive_tracers.keys()

In [ ]:
FigTracers = vis.Figure(figsize=(1200,400))

# Show single colour
# Fig.Points(Model.swarm, colour='gray', opacity=0.5, discrete=True, 
#            fn_mask=materialFilter, fn_size=2.0, colourBar=False)

# Show all glory
FigTracers.Points(Model.swarm, fn_colour=Model.materialField, 
           fn_mask=materialFilter, opacity=0.5, fn_size=2.0)

def get_show_tracer(name, colours):
    t = Model.passive_tracers.get(name)
    if not t: raise RuntimeError("ERROR: fine tracer called ", name)
    
    FigTracers.Points(t, t.variables[-1],fn_size=2.,
                      colours=colours,opacity=0.5,colourBar=False)
    
if dim == 3:
    
    get_show_tracer(name='orp', colours="#22BBBB #335588")
    get_show_tracer(name='slab', colours="Gray40 Goldenrod")
    get_show_tracer(name='cont', colours="#335588 #22BBBB")
    get_show_tracer(name='arc', colours="Goldenrod Grey41")
    get_show_tracer(name='buoy', colours="#335588 #335588")

# Rotate camera angle
FigTracers.script(camera)

# Render in notebook
# FigTracers.show()

In [ ]:
Model.minViscosity = dimensionalise(1., u.Pa * u.sec)
Model.maxViscosity = dimensionalise(1e5, u.Pa * u.sec)

In [ ]:
Model.set_velocityBCs( left=[0.,None,None], right=[0.,None,None],
                       bottom=[None,0.,None], top=[None,0.,None],
                       front=[None,None,0.], back=[None,None,0.])

In [ ]:
Model.init_model()

In [ ]:
# figViscosity = vis.Figure(figsize=figsize, axis=True)
# figViscosity.append( vis.oGEO.rcParams[“default.outputs”]bjects.Points(swarm, viscosityFn, colours='dem1', fn_size=2., logScale=True) )
# if dim == 3:
#     figViscosity.script(camera)
# figViscosity.show()

In [ ]:
Fig = vis.Figure(figsize=(1200,400))

# Show all glory
Fig.Points(Model.swarm, fn_colour=Model.densityField, 
           fn_mask=materialFilter, opacity=0.5, fn_size=2.0)

# Rotate camera angle
Fig.script(camera)

# Render in notebook
# Fig.show()

In [ ]:
if rank == 0:
    with open(outputPath+'FrequentOutput.dat','a') as f:
         f.write('step\t time(Myr)\t Vrms(cm/yr)\n')
            
def post_solve_hook():
    vrms = Model.stokes_SLE.velocity_rms()
    step = Model.step
    time = Model.time.to(u.megayear)
    
    if rank == 0:
        with open(outputPath+'FrequentOutput.dat','a') as f:
             f.write(f"{step}\t{time:5e}\t{vrms:5e}")
        
Model.post_solve_functions["Measurements"] = post_solve_hook

In [ ]:
## We can test different solvers by uncommentting this section
# solver = Model.solver
# # System level solver options
# solver.options.main.Q22_pc_type = "uwscale"
# solver.options.main.ksp_k2_type = "GMG"
# solver.options.main.ksp_type    = "bsscr"
# solver.options.main.pc_type     = "none"
# solver.options.main.penalty     = 50.
# #solver.options.main.list()

# # Schur complement solver options
# solver.options.scr.ksp_rtol = 1.0e-3 
# solver.options.scr.ksp_type = "fgmres"
# #solver.options.main.list()

# # Inner solve (velocity), A11 options
# solver.options.A11.ksp_rtol = 1.0e-4
# solver.options.A11.ksp_type = "fgmres"
# #solver.options.A11.list()
# solver.print_petsc_options()

In [ ]:
## debugging code to generate initial fields for viscosity and density ##
# fields output used to analyse initial setup

jeta = Model.add_submesh_field(name="cell_vis", nodeDofCount=1)
jrho = Model.add_submesh_field(name="cell_rho", nodeDofCount=1)

GEO.rcParams["default.outputs"].append("cell_vis")
GEO.rcParams["default.outputs"].append("cell_rho")

subMesh = Model.mesh.subMesh
jeta.data[:] = Model._viscosityFn.evaluate(subMesh)
jrho.data[:] = Model._densityFn.evaluate(subMesh)

In [ ]:
Model.run_for(nstep=0, checkpoint_interval=1)

In [ ]:
# Model.run_for(nstep=0, checkpoint_interval=1)